In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR, StepLR, CosineAnnealingLR, CyclicLR
import wandb

In [2]:
class MyNetwork(torch.nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2,  stride=2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
model = MyNetwork()
optimizer = optim.Adam(model.parameters(), lr=0.01)

lr_schedulers = {
    'ExponentialLR': ExponentialLR(optimizer=optimizer, gamma=0.95),
    'StepLR': StepLR(optimizer=optimizer, step_size=10),
    'CosineAnnealingLR': CosineAnnealingLR(optimizer=optimizer, T_max=10),
    'CyclicLR': CyclicLR(optimizer=optimizer, base_lr=0.01, max_lr=0.1, mode='triangular2', cycle_momentum=False)
}

**Adjust Learning Rate**

Phan Dinh Truong 20214937 $\to$ Choose method:

- [0] Exponential LR
- [3] Cyclic LR *(mode="triangle2")*

In [4]:
wandb.init(project='LearningRate', name='learning_rate.ipynb')

for scheduler_name, scheduler in lr_schedulers.items():
    if scheduler_name == 'ExponentialLR' or scheduler_name == 'CyclicLR':
        wandb.run.name = scheduler_name

        losses = []
        for epoch in range(50):
            loss = torch.rand(1)
            losses.append(loss.item())

            scheduler.step()

            wandb.log({"learning_rate": optimizer.param_groups[0]['lr'], "loss": loss.item()})
        
        wandb.log({f"{scheduler_name}_loss": losses})

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: